In [1]:
import os

In [3]:
def realcase(path):
    """
    Returns the case-corrected path if it exists, otherwise returns "NotExist".

    Args:
        path (str): The path to be case-corrected.

    Returns:
        str: The case-corrected path or "NotExist" if the path does not exist.
    """
    # Handle empty or None path
    if not path:
        return "NotExist"
    
    # Normalize the path
    path = os.path.normpath(path)
    dirname, basename = os.path.split(path)

    # Base case: if we've reached the root
    if dirname == path:
        return dirname if os.path.exists(dirname) else "NotExist"
    
    # Recursively get the case-corrected dirname
    dirname = realcase(dirname)
    
    # If parent directory doesn't exist, return NotExist
    if dirname == "NotExist":
        return "NotExist"
    
    basename = os.path.normcase(basename)

    try:
        for child in os.listdir(dirname):
            if os.path.normcase(child) == basename: 
                return os.path.join(dirname, child)
    except (FileNotFoundError, PermissionError, OSError):
        return "NotExist"
    
    return "NotExist"

In [5]:
def check_file_existence(base_folder, ctl_file): 
    """
    Checks if all files listed in the CTL file exist in the repository.
    
    Args:
        base_folder (str): Path to the main repository folder
        ctl_file (str): Path to the .ctl file with the list of files to check
        
    Returns:
        bool: True if all files exist, False otherwise
    """
    try:
        with open(ctl_file, 'r', encoding='utf-8') as f:
            paths = f.read().splitlines()
        
        missing_files = []
        existing_files = []
        
        for path in paths:
            path = path.strip()
            if path and not path.startswith('#'):  # Ignore empty lines and comments
                full_path = os.path.join(base_folder, path)
                # Use realcase function to check the actual path
                real_path = realcase(full_path)
                
                if real_path != "NotExist":
                    existing_files.append(f"✓ {path} -> {real_path}")
                else:
                    missing_files.append(f"✗ {path}")
        
        # Display results
        print(f"Checking files from {ctl_file}:")
        print(f"Base folder: {base_folder}")
        print("-" * 50)
        
        if existing_files:
            print("Existing files:")
            for file in existing_files:
                print(file)
        
        if missing_files:
            print("\nMISSING files:")
            for file in missing_files:
                print(file)
        
        print(f"\nSummary:")
        print(f"Existing: {len(existing_files)}")
        print(f"Missing: {len(missing_files)}")
        
        return len(missing_files) == 0
        
    except FileNotFoundError:
        print(f"Error: CTL file not found: {ctl_file}")
        return False
    except Exception as e:
        print(f"Error during check: {e}")
        return False

In [6]:
def generate_ctl_file(base_folder, output_ctl_file, folders_to_include=None, file_extensions=None):
    """
    Generates a CTL file with all files from specified folders.
    
    Args:
        base_folder (str): Path to the main repository folder
        output_ctl_file (str): Path to the CTL file to be created
        folders_to_include (list): List of folders to include (e.g., ['Tbl', 'Pro', 'Trg'])
        file_extensions (list): List of file extensions to include (e.g., ['.tbl', '.pro', '.trg'])
        
    Returns:
        list: List of found files
    """
    if folders_to_include is None:
        folders_to_include = ['Tbl', 'Pro', 'Trg', 'Fnc', 'Vw', 'Seq', 'Idx']
    
    if file_extensions is None:
        file_extensions = ['.tbl', '.pro', '.trg', '.fnc', '.vw', '.seq', '.idx', '.sql']
    
    files_found = []
    
    try:
        for folder in folders_to_include:
            folder_path = os.path.join(base_folder, folder)
            if os.path.exists(folder_path):
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        if any(file.lower().endswith(ext.lower()) for ext in file_extensions):
                            # Create relative path from base_folder
                            relative_path = os.path.relpath(os.path.join(root, file), base_folder)
                            # Replace backslashes with forward slashes for consistency
                            relative_path = relative_path.replace('\\', '/')
                            files_found.append(relative_path)
        
        # Sort files alphabetically
        files_found.sort()
        
        # Write to CTL file
        with open(output_ctl_file, 'w', encoding='utf-8') as f:
            f.write(f"# CTL file generated automatically\n")
            f.write(f"# File: {os.path.basename(output_ctl_file)}\n")
            f.write(f"# Base folder: {base_folder}\n")
            f.write(f"# Files found: {len(files_found)}\n")
            f.write(f"#\n")
            for file_path in files_found:
                f.write(f"{file_path}\n")
        
        print(f"Generated CTL file: {output_ctl_file}")
        print(f"Found {len(files_found)} files in folders: {', '.join(folders_to_include)}")
        print(f"Extensions: {', '.join(file_extensions)}")
        
        return files_found
        
    except Exception as e:
        print(f"Error generating CTL file: {e}")
        return []

# CTL File Management Script

## Description
This script is used to manage CTL (Control files) used for database releases via Harness.

## Repository Structure
Typical database repository structure:
```
project-repo/
├── Tbl/           # Tables (.tbl)
├── Pro/           # Procedures (.pro)
├── Trg/           # Triggers (.trg)
├── Fnc/           # Functions (.fnc)
├── Vw/            # Views (.vw)
├── Seq/           # Sequences (.seq)
├── Idx/           # Indexes (.idx)
└── release.ctl    # Control file with list of files to release
```

## Script Functions

### 1. `realcase(path)`
- Returns the actual path of a file with correct case
- Useful on Windows systems where case sensitivity may differ

### 2. `check_file_existence(base_folder, ctl_file)`
- Checks if all files listed in the CTL file exist in the repository
- Displays list of existing and missing files
- Returns `True` if all files exist

### 3. `generate_ctl_file(base_folder, output_ctl_file, folders_to_include, file_extensions)`
- Automatically generates a CTL file with all files from specified folders
- Useful for creating complete CTL before release

## Usage
1. **Before release**: Check if all files from CTL exist in repo
2. **Creating CTL**: Generate CTL file with all objects
3. **Validation**: Ensure no files have been missed

In [12]:
# USAGE EXAMPLES AND TESTING

# Configuration paths - CHANGE TO YOUR PATHS
# base_folder = r"C:\path\to\your\database-repo"  # Path to main repo folder
base_folder = r"C:\Users\MM\Desktop\python-scripts\test-repo"  # Path to main repo folder
# ctl_file = r"C:\path\to\your\release.ctl"       # Path to CTL file to check
ctl_file = r"C:\Users\MM\Desktop\python-scripts\test-repo\test_release.ctl"       # Path to CTL file to check

# Example 1: Check if all files from CTL exist
print("=== EXAMPLE 1: Checking files from CTL ===")
if os.path.exists(base_folder) and os.path.exists(ctl_file):
    result = check_file_existence(base_folder, ctl_file)
    if result:
        print("✓ All files exist!")
    else:
        print("✗ Some files are missing!")
else:
    print("Check paths - folder or CTL file does not exist")

print("\n" + "="*60 + "\n")

# Example 2: Generate new CTL file with all objects
print("=== EXAMPLE 2: Generating full CTL ===")
output_ctl = os.path.join(os.path.dirname(base_folder), "full_release.ctl")
if os.path.exists(base_folder):
    files = generate_ctl_file(
        base_folder=base_folder,
        output_ctl_file=output_ctl,
        folders_to_include=['Tbl', 'Pro', 'Trg', 'Fnc', 'Vw'],
        file_extensions=['.tbl', '.pro', '.trg', '.fnc', '.vw', '.sql']
    )
    if files:
        print(f"First 5 files in CTL:")
        for i, file in enumerate(files[:5]):
            print(f"  {i+1}. {file}")
        if len(files) > 5:
            print(f"  ... and {len(files)-5} more")
else:
    print("Check base_folder path")

print("\n" + "="*60 + "\n")

# Example 3: Check specific file
print("=== EXAMPLE 3: Checking specific file ===")
test_file = os.path.join(base_folder, "Tbl", "example_table.tbl")
real_path = realcase(test_file)
print(f"Checked file: {test_file}")
print(f"Real path: {real_path}")
if real_path != "NotExist":
    print("✓ File exists")
else:
    print("✗ File does not exist")

=== EXAMPLE 1: Checking files from CTL ===
Checking files from C:\Users\MM\Desktop\python-scripts\test-repo\test_release.ctl:
Base folder: C:\Users\MM\Desktop\python-scripts\test-repo
--------------------------------------------------
Existing files:
✓ Tbl/users.tbl -> C:\Users\MM\Desktop\python-scripts\test-repo\Tbl\users.tbl
✓ Tbl/products.tbl -> C:\Users\MM\Desktop\python-scripts\test-repo\Tbl\products.tbl
✓ Pro/get_user.pro -> C:\Users\MM\Desktop\python-scripts\test-repo\Pro\get_user.pro
✓ Pro/update_product.pro -> C:\Users\MM\Desktop\python-scripts\test-repo\Pro\update_product.pro

MISSING files:
✗ Trg/user_audit.trg

Summary:
Existing: 4
Missing: 1
✗ Some files are missing!


=== EXAMPLE 2: Generating full CTL ===
Generated CTL file: C:\Users\MM\Desktop\python-scripts\full_release.ctl
Found 4 files in folders: Tbl, Pro, Trg, Fnc, Vw
Extensions: .tbl, .pro, .trg, .fnc, .vw, .sql
First 5 files in CTL:
  1. Pro/get_user.pro
  2. Pro/update_product.pro
  3. Tbl/products.tbl
  4. Tbl/

# Testing Instructions

## Test Environment Setup

### 1. Create test structure
```
test-repo/
├── Tbl/
│   ├── users.tbl
│   └── products.tbl
├── Pro/
│   ├── get_user.pro
│   └── update_product.pro
├── Trg/
│   └── user_audit.trg
└── test_release.ctl
```

### 2. Sample test_release.ctl content
```
# Test release CTL
Tbl/users.tbl
Tbl/products.tbl
Pro/get_user.pro
Pro/update_product.pro
Trg/user_audit.trg
```

### 3. Testing steps

1. **Change paths in examples** - set `base_folder` to path to `test-repo`
2. **Run cells sequentially** - first import, then functions, finally examples
3. **Test different scenarios**:
   - Check existing files
   - Check non-existing files
   - Generate full CTL
   - Compare results

## Common Use Cases

### Before release to TEST/PROD
```python
# Check if all files from CTL exist
result = check_file_existence("C:\\repo\\my-db-project", "C:\\repo\\my-db-project\\release_v1.2.ctl")
```

### Generate CTL for all objects
```python
# Generate full CTL with all files
generate_ctl_file(
    base_folder="C:\\repo\\my-db-project",
    output_ctl_file="C:\\repo\\my-db-project\\full_release.ctl",
    folders_to_include=['Tbl', 'Pro', 'Trg', 'Fnc', 'Vw']
)
```

### Verify specific file
```python
# Check if specific file exists with correct case
real_path = realcase("C:\\repo\\my-db-project\\Tbl\\USER_TABLE.tbl")
```